In [1]:
import CS_Indicators as CS
from brix import Indicator, Handler
import PreCompOsmNet
import Simulation
import geopandas as gpd
import pandas as pd

In [2]:
table_name='providence_test'

### Load the Saved Data

In [3]:
geogrid=gpd.read_file('tables/{}/geogrid.geojson'.format(table_name))
zones=gpd.read_file('tables/{}/zones.geojson'.format(table_name))
zones['GEOID']=zones['GEOID'].astype(int)
zones=zones.set_index('GEOID')
simpop_df=pd.read_csv('tables/{}/simpop_df.csv'.format(table_name))

### Create the mobility system

In [4]:
networks, mode_dicts, route_lengths=PreCompOsmNet.create_2_scale_osmnx_network(
    zones.loc[zones['sim_area']], zones.loc[zones['model_area']],
    add_modes=[{'name': 'walk', 'speed': 4800/3600},
               {'name': 'cycle', 'speed': 14000/3600},
               {'name': 'pt', 'speed': 25000/3600}])
modes={mode: Simulation.Mode(mode_dicts[mode]) for mode in mode_dicts}
mob_sys=Simulation.MobilitySystem(modes=modes, networks=networks)

getting internal roads
Buffer : 200
Doesnt contain external link types
Buffer : 400
getting external roads
	 Simplified from 2143 to 1891 edges and 929 to 750 nodes
	 Pre-computing paths


In [5]:
import numpy as np

class Default_Mode_Choice_model():
    def __init__(self):
        pass
    
    def predict_modes(self, all_trips_df):
        options=['drive', 'cycle', 'walk', 'pt']
        all_trips_df['mode']='drive'
        ind_u1500=all_trips_df['route_distance']<1500
        ind_1500_5k=((all_trips_df['route_distance']>1500)&(all_trips_df['route_distance']<5000))
        ind_5k_10k=((all_trips_df['route_distance']>5000)&(all_trips_df['route_distance']<10000))
        ind_10kplus=all_trips_df['route_distance']>10000

        all_trips_df.loc[ind_u1500==True, 'mode']=np.random.choice(
            options, size=sum(ind_u1500), 
            replace=True, p=[0.1, 0.2, 0.6, 0.1])

        all_trips_df.loc[ind_1500_5k==True, 'mode']=np.random.choice(
            options, size=sum(ind_1500_5k), 
            replace=True, p=[0.25, 0.15, 0.3, 0.3])

        all_trips_df.loc[ind_5k_10k==True, 'mode']=np.random.choice(
            options, size=sum(ind_5k_10k), 
            replace=True, p=[0.6, 0.04, 0.01, 0.35])

        all_trips_df.loc[ind_10kplus==True, 'mode']=np.random.choice(
            options, size=sum(ind_10kplus), 
            replace=True, p=[0.9, 0.01, 0.01, 0.08])
        return all_trips_df

mode_descriptions = [{"name": 'drive',
                    'color': "#e41a1d"},
                     {"name": 'cycle',
                    'color': "#377eb8"},
                     {"name": 'walk',
                    'color': "#4daf4a"},
                     {"name": 'pt',
                    'color': "#ffff33"},
                    ]

### Create and run the Cityscope model

In [6]:
H=Handler(table_name=table_name)
H.reset_geogrid_data()

https://cityio.media.mit.edu/api/tables/list/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/GEOGRID/ Attempt: 0
Proceeding without reviewing GEOGRIDDATA
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
GEOGRIDDATA successfully updated: 6i37RGg4CAEQAvtSa6fnnPaPCRjYMxc65r5q6Spcd5XY


In [7]:
# d=CS.Density_Indicator(zones=zones)
# p=CS.Proximity_Indicator(zones=zones, geogrid=geogrid)


m=CS.Mobility_indicator(zones, geogrid, table_name, simpop_df, mob_sys,
#                        mode_choice_model=new_mode_choice_model,
#                        mode_descriptions=mode_descriptions,
                       route_lengths=route_lengths)
H.add_indicators([
#     d, p, 
    m
])

# H.add_indicator(m)

Init simulation


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/CS_Indicators.py:599: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids=grid_zones['geometry'].centroid


Getting internal nodes
Finding closest nodes to every zone centroid
Computing zone-zone dist mat


/Users/doorleyr/anaconda3/envs/brix/lib/python3.9/site-packages/osmnx/distance.py:426: UserWarning: The `get_nearest_nodes` function has been deprecated and will be removed in a future release. Use the more efficient `distance.nearest_nodes` instead.
  warnings.warn(msg)


https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.274961
	 trips post took 0:00:01.631541


In [8]:
geogrid_data=H.get_geogrid_data()
m.return_indicator(geogrid_data)

https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.350264
	 trips post took 0:00:01.808504


{'name': 'Active Mobility',
 'raw_value': 0.36,
 'value': 0.36,
 'ref_value': 0.1,
 'viz_type': 'radar'}

In [9]:
H.listen()

Table URL: https://cityscope.media.mit.edu/CS_cityscopeJS/?cityscope=providence_test
Testing indicators
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.355578
	 trips post took 0:00:01.927393
Performing initial update
Updating geogrid_data
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
Updating table with hash: 6i37RGg4CAEQAvtSa6fnnPaPCRjYMxc65r5q6Spcd5XY
https://cityio.media.mit.edu/api/table/providence_test/GEOGRIDDATA/ Attempt: 0
Starting MM Update
Schedules and locations
OD
Trip table
Route table
DeckGL


/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


Post ABM: <Response [200]>
Post OD: <Response [200]>
Finished MM Update
	 calculation took 0:00:00.243582
	 trips post took 0:00:01.607559
Done with update
Local grid hash: 6i37RGg4CAEQAvtSa6fnnPaPCRjYMxc65r5q6Spcd5XY
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Att

https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_te

/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:406: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['node_path']=routes['node_path']
/Volumes/GoogleDrive/My Drive/Media Lab/NuevoScope/CS_Spatial_Modules/Simulation.py:407: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  route_table_this_mode['attributes']=routes['attributes']


DeckGL
Post ABM: <Response [200]>
Post OD: <Response [400]>
Finished MM Update
	 calculation took 0:00:05.605745
	 trips post took 0:00:01.493063
Done with update
Local grid hash: EzfMs9fZ9kjep8S5CFNB8LhhDTUZ1MxrnnJiGmGdzTXF
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hashes/ Attempt: 0
https://cityio.media.mit.edu/api/table/providence_test/meta/hash

KeyboardInterrupt: 